교차 검증과 그리드 서치

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

검증 세트 만들기

In [2]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

검증 세트로 평가하기

In [3]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
dt.score(sub_input, sub_target), dt.score(val_input, val_target)

(0.9971133028626413, 0.864423076923077)

교차 검증

In [5]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.02868891, 0.02315974, 0.0214982 , 0.02469516, 0.02583408]), 'score_time': array([0.01260662, 0.0017035 , 0.00165486, 0.00153947, 0.00161839]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [6]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


분할기를 사용한 교차 검증

In [7]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [8]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # 10개의 스플릿, 섞기 허용
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


그리드 서치

In [9]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [10]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [11]:
ds = gs.best_estimator_
print(ds.score(train_input, train_target))

0.9615162593804117


In [12]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [13]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


더 복잡한 탐색

In [14]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
         }

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [15]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


랜덤 서치

In [17]:
!pip install scipy

In [18]:
from scipy.stats import uniform # 균등분포
from scipy.stats import randint # 정규 분포에서 정수 뽑기

In [19]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)
         }

In [20]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [21]:
np.max(gs.cv_results_['mean_test_score'])

0.8695428296438884

In [23]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


확률 분포 선택

In [25]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([4, 8, 1, 8, 7, 2, 0, 2, 9, 0])

In [26]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 99,  97, 102,  99,  92, 117,  99,  94, 110,  91]))

In [27]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.15827957, 0.32023659, 0.5147389 , 0.00159831, 0.4735059 ,
       0.47239054, 0.30066738, 0.31018282, 0.77144774, 0.37861898])